# The aim of this notebook is to answer following questions on the given dataset.

Using data in 'data/mondial_database.xml'

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

# import libraries

In [386]:
from xml.etree import ElementTree as ET
import pandas as pd

# load data data

In [387]:
document_tree = ET.parse("data/mondial_database.xml",)

In [388]:
document_root = document_tree.getroot()

# get an overview of the data

To get an overview about the structure of the xml document I used a texteditor.

# 1. 10 countries with the lowest infant mortality rates

In [389]:
country_infant_mort = pd.DataFrame(columns=["country", "infant_mortality"])

for country in document_root.getiterator("country"):
    # ignore countrys without a infant_mortality tag
    if country.find("infant_mortality") is None:
        continue
    
    name = country.find("name").text
    mort_rate = country.find("infant_mortality").text
    
    country_infant_mort = country_infant_mort.append(
        {
            "country": name, 
            "infant_mortality": mort_rate
        }, ignore_index=True)
    
country_infant_mort.sort_values("infant_mortality", ascending=True)[0:10]

,country,infant_mortality
36,Monaco,1.81
28,Romania,10.16
142,Fiji,10.2
63,Brunei,10.48
124,Grenada,10.5
221,Mauritius,10.59
116,Panama,10.7
227,Seychelles,10.77
94,United Arab Emirates,10.92
105,Barbados,10.93


# 2. 10 cities with the largest population

In [390]:
# assumption is to select to most recent data of population years 
# so I take the latest measurement as metric

# dataframe for the population measurement with year and country 
df_city_pop = pd.DataFrame(columns=["city", "year", "population"])

# iterate over xml - tree and extract
# city, year and population
for country in document_root.getiterator("country"):
    for city in country.iter("city"):
        city_name = city.find("name").text
        
        if city.findall("population") is None:
            continue;
        
        for pop in city.findall("population"):
            year = pop.attrib["year"]
            population = pop.text
            
            df_city_pop = df_city_pop.append(
            {
                "city": city_name,
                "year": year,
                "population": population
            }, ignore_index=True)
            
# check the data
df_city_pop.head()

,city,year,population
0,Tirana,1987,192000
1,Tirana,1990,244153
2,Tirana,2011,418495
3,Shkodër,1987,62000
4,Shkodër,2011,77075


In [391]:
# copy dataframe to save it for changes
df_city_pop_c = df_city_pop.copy()

# dataframe for the latest population measurement with year and city 
df_latest_city_pop = pd.DataFrame(columns=["city", "year", "population"])

# change population column to numeric
df_city_pop_c["population"] = pd.to_numeric(df_city_pop_c["population"])

# select all unique cities
cities = df_city_pop["city"].unique()

# assamble all cities with latest population measurement in one dataframe
for city in cities:
    city_pop = df_city_pop_c[df_city_pop_c.city == city]
    latest_measurement = city_pop[city_pop["year"] == city_pop["year"].max()]
    df_latest_city_pop = df_latest_city_pop.append(latest_measurement)

# sort values by population in descending order and take the first 10 elements
df_latest_city_pop.sort_values("population", ascending=False)[0:10]

,city,year,population
3750,Shanghai,2010,22315474.0
2607,Istanbul,2012,13710512.0
4303,Mumbai,2011,12442373.0
1546,Moskva,2013,11979529.0
3746,Beijing,2010,11716620.0
8208,São Paulo,2010,11152344.0
3754,Tianjin,2010,11090314.0
3364,Guangzhou,2010,11071424.0
4399,Delhi,2011,11034555.0
3371,Shenzhen,2010,10358381.0


## 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [392]:
# dataframe for the latest population measurement with year and country 
df_country_pop = pd.DataFrame(columns=["country", "year", "population"])

d = []
# iterate over xml - tree and extract
# country, year and population
for country in document_root.getiterator("country"):
    for element in list(country):
        if element.tag == "population":
            country_name = country.find("name").text
            year = element.attrib["year"]
            population = element.text
               
            df_country_pop = df_country_pop.append({
                "country": country_name,
                "year": year,
                "population": population
            }, ignore_index=True)

df_country_pop

,country,year,population
0,Albania,1950,1214489
1,Albania,1960,1618829
2,Albania,1970,2138966
3,Albania,1980,2734776
4,Albania,1990,3446882
5,Albania,1997,3249136
6,Albania,2000,3304948
7,Albania,2001,3069275
8,Albania,2011,2800138
9,Greece,1861,1096810


Extract the latest measurment for each country

In [393]:
# copy dataframe to save it for changes
df_country_pop_c = df_country_pop.copy()

# dataframe for the latest population measurement with year and city 
df_latest_country_pop = pd.DataFrame(columns=["country", "year", "population"])

# change population column to numeric
df_country_pop_c["population"] = pd.to_numeric(df_country_pop_c["population"])

# select all unique countries
countries = df_country_pop["country"].unique()

# assamble all cities with latest population measurement in one dataframe
for country in countries:
    country_pop = df_country_pop_c[df_country_pop_c.country == country]
    latest_measurement = country_pop[country_pop["year"] == country_pop["year"].max()]
    df_latest_country_pop = df_latest_country_pop.append(latest_measurement)
df_latest_country_pop.head()

,country,year,population
8,Albania,2011,2800138.0
23,Greece,2011,10816286.0
34,Macedonia,2011,2059794.0
42,Serbia,2011,7120666.0
52,Montenegro,2011,620029.0


In [394]:
# create dataframe for all distinct ethnicgroups
df_ethnic_groups = pd.DataFrame(columns=["ethnicgroup", "population"])

for country in document_root.getiterator("country"):
    for element in list(country):
        if element.tag == "ethnicgroup":
            df_ethnic_groups = df_ethnic_groups.append({
                "ethnicgroup": element.text,
                "population": 0.0
            }, ignore_index=True)
df_ethnic_groups = df_ethnic_groups.drop_duplicates("ethnicgroup")
df_ethnic_groups.head()

,ethnicgroup,population
0,Albanian,0.0
1,Greek,0.0
3,Macedonian,0.0
5,Turkish,0.0
6,Gypsy,0.0


Get fraction of ethnic groups for each country

In [395]:
# dataframe for fraction of each ethnic group in country
df_ethnic_groups_fraction = pd.DataFrame(columns=["country", "ethnicgroup", "fraction"])

for country in document_root.getiterator("country"):
    for element in list(country):
        if element.tag == "ethnicgroup":
            df_ethnic_groups_fraction = df_ethnic_groups_fraction.append({
                "country": country.find("name").text,
                "ethnicgroup": element.text,
                "fraction": element.attrib["percentage"]
            }, ignore_index=True)
            
# convert fraction to numeric
df_ethnic_groups_fraction["fraction"] = pd.to_numeric(df_ethnic_groups_fraction["fraction"])

# get fraction between 0 and 1
df_ethnic_groups_fraction["fraction"] = df_ethnic_groups_fraction["fraction"] / 100
df_ethnic_groups_fraction.head()

,country,ethnicgroup,fraction
0,Albania,Albanian,0.950
1,Albania,Greek,0.030
2,Greece,Greek,0.930
3,Macedonia,Macedonian,0.642
4,Macedonia,Albanian,0.252


Iterate over df_ethnic_groups_fraction and df_country_population and calculate the overall

In [396]:
df_ethnic_groups_c = df_ethnic_groups.copy()

for country in countries:
    # population for country
    population = df_latest_country_pop[df_latest_country_pop.country == country].population.item()
    
    # get ethnicgroups for each country
    ethnic_groups = df_ethnic_groups_fraction[df_ethnic_groups_fraction.country == country]
    
    for index, row in ethnic_groups.iterrows():
        population_fraction = row["fraction"]
        ethnic_group = row["ethnicgroup"]
        absolute_popluation = population * population_fraction
        temp_pop = df_ethnic_groups_c[df_ethnic_groups_c.ethnicgroup == ethnic_group].population.item()
        
        df_ethnic_groups_c.loc[df_ethnic_groups_c.ethnicgroup == ethnic_group, "population"] += absolute_popluation
        
df_ethnic_groups_c.sort_values("population", ascending=False)[0:10]

,ethnicgroup,population
176,Han Chinese,1.245059e+09
221,Indo-Aryan,8.718156e+08
269,European,4.948722e+08
24,African,3.183251e+08
220,Dravidian,3.027137e+08
324,Mestizo,1.577344e+08
212,Bengali,1.467769e+08
66,Russian,1.318570e+08
299,Japanese,1.265342e+08
226,Malay,1.219936e+08


## 4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

Create a dictionary for the country - country_code realtion

In [397]:
df_country_codes = pd.DataFrame(columns=["country_code", "country"])

for country in document_root.iterfind("country"):
    df_country_codes = df_country_codes.append({
        "country_code": country.attrib["car_code"],
        "country": country.find("name").text
    }, ignore_index=True)
    
dic_country_codes = pd.Series(df_country_codes.country.values, index=df_country_codes.country_code).to_dict()

### a) longest river

In [398]:
# dataframe for all rivers with country_code and length
df_river = pd.DataFrame(columns=["river_name", "country_code", "length"])

for river in document_root.iterfind("river"):
    if (river.find("length") is not None) and (river.find("located") is not None):
        df_river = df_river.append({
            "river_name": river.find("name").text,
            "country_code": river.find("located").attrib["country"],
            "length": river.find("length").text
        }, ignore_index=True)
        
df_river_longest = df_river[df_river.length == df_river.length.max()]

In [399]:
# dataframe to store longest river with country and length
df_river_longest_country = pd.DataFrame(columns=["river_name", "country", "length"])

# get country name for country_code
country_name = dic_country_codes.get(df_river_longest.iloc[0]["country_code"])

df_river_longest_country = df_river_longest_country.append({
    "river_name": df_river_longest.iloc[0]["river_name"],
    "country": country_name,
    "length": df_river_longest.iloc[0]["length"]
}, ignore_index=True) 


df_river_longest_country

,river_name,country,length
0,Selenge,Russia,992


### b) largest lake

In [400]:
# assumption for the size of the largest lake I take the area property

# dataframe for all lakes with country_code and length
df_lake = pd.DataFrame(columns=["lake_name", "country_code", "area"])

for lake in document_root.iterfind("lake"):
    if (lake.find("area") is not None) and (lake.find("located") is not None):
        df_lake = df_lake.append({
            "lake_name": lake.find("name").text,
            "country_code": lake.find("located").attrib["country"],
            "area": lake.find("area").text
        }, ignore_index=True)

df_lake_largest = df_lake[df_lake.area == df_lake.area.max()]

In [401]:
# dataframe to store lake with country_name and area
df_lake_largest_country = pd.DataFrame(columns=["lake_name", "country", "area"])

# get country name for country_code
country_name = dic_country_codes.get(df_lake_largest.iloc[0]["country_code"])

df_lake_largest_country = df_lake_largest_country.append({
    "lake_name": df_lake_largest.iloc[0]["lake_name"],
    "country": country_name,
    "area": df_lake_largest.iloc[0]["area"]
}, ignore_index=True)
df_lake_largest_country

,lake_name,country,area
0,Fort Peck Lake,United States,981


### c) airport at highest elevation

In [402]:
# dataframe for all airports with elevation and country_code 
df_air = pd.DataFrame(columns=["airport_name", "country_code", "elevation"])

for airport in document_root.iterfind("airport"):
    df_air = df_air.append({
        "airport_name": airport.find("name").text,
        "country_code": airport.attrib["country"],
        "elevation": airport.find("elevation").text
    }, ignore_index=True)
    
# convert elevation to numeric
df_air["elevation"] = pd.to_numeric(df_air["elevation"])

# select airport with highest elevation
df_air_highest = df_air[df_air.elevation == df_air.elevation.max()]

In [403]:
# dataframe to store airport with country_name and elevation
df_air_highest_country = pd.DataFrame(columns=["airport_name", "country", "elevation"])

# get country name for country_code
country_name = dic_country_codes.get(df_air_highest.iloc[0]["country_code"])

df_air_highest_country = df_air_highest_country.append({
        "airport_name": df_air_highest.iloc[0]["airport_name"],
        "country": country_name,
        "elevation": df_air_highest.iloc[0]["elevation"]
    }, ignore_index=True)
df_air_highest_country

,airport_name,country,elevation
0,El Alto Intl,Bolivia,4063.0
